In [1]:
import pandas as pd
import numpy as np

from hydra.utils import instantiate
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra

from data_pipeline.pipelinesteps import data_splitter
from model.modelorchastrator import ModelOrchestrator
from data_pipeline.dataset import Dataset
from evaluate.metric_eval import ModelEvaluator

(CVXPY) Mar 21 11:34:54 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 21 11:34:54 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
 2024-03-21 11:34:54,930 -  INFO  	: my_logger.custom_logger - Logger is set up.


In [2]:
# Create config
GlobalHydra.instance().clear()
initialize(config_path="../../", version_base=None)
cfg = compose(config_name="config")

In [3]:
# Create Dataset
df = pd.read_feather("../../data/parquet_files/train/processed_train.feather")
df.pop('date_decision')
data_pipeline = instantiate(cfg.data_pipeline)
df = data_pipeline.apply(df)  

dataset = Dataset(data=df, data_splitter=data_splitter, target_column='target')

In [5]:
model_orchestrator = ModelOrchestrator(cfg)

model = model_orchestrator.modelpipeline
ME = ModelEvaluator(model[-1], ['roc_auc_ovr', 'f1'])

In [9]:
df['education_927M_last']

0          a55475b1
1          a55475b1
2          a55475b1
3          a55475b1
4          a55475b1
             ...   
1526654    a55475b1
1526655    a55475b1
1526656    a55475b1
1526657    a55475b1
1526658    a55475b1
Name: education_927M_last, Length: 1526659, dtype: object

In [ ]:
model.fit(dataset.X_train, dataset.y_train)

c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


Binning 0.142 GB of training data: 0.489 s
Binning 0.016 GB of validation data: 0.013 s
Fitting gradient boosted rounds:
[1/10] 1 tree, 100 leaves, max depth = 13, train loss: 0.68206, val loss: 0.68303, in 0.204s
[2/10] 1 tree, 100 leaves, max depth = 13, train loss: 0.67292, val loss: 0.67492, in 0.175s
[3/10] 1 tree, 100 leaves, max depth = 12, train loss: 0.66524, val loss: 0.66821, in 0.208s
[4/10] 1 tree, 100 leaves, max depth = 13, train loss: 0.65878, val loss: 0.66268, in 0.138s
[5/10] 1 tree, 100 leaves, max depth = 14, train loss: 0.65328, val loss: 0.65796, in 0.122s
[6/10] 1 tree, 100 leaves, max depth = 12, train loss: 0.64868, val loss: 0.65397, in 0.120s
[7/10] 1 tree, 100 leaves, max depth = 14, train loss: 0.64464, val loss: 0.65062, in 0.130s
[8/10] 1 tree, 100 leaves, max depth = 14, train loss: 0.64113, val loss: 0.64777, in 0.147s
[9/10] 1 tree, 100 leaves, max depth = 15, train loss: 0.63805, val loss: 0.64528, in 0.153s
[10/10] 1 tree, 100 leaves, max depth = 16

In [ ]:
model.transform_without_predictor(dataset.X_train)

c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


,index,case_id,cardtype_51L_last,credtype_322L_last,numberofcontrsvalue_358L_max,nominalrate_281L_min,classificationofcontr_400M_mode,totaldebtoverduevalue_178A_max,overdueamount_31A_max,residualamount_488A_max,collater_valueofguarantee_876L_mean,pmts_overdue_1140A_max,pmts_overdue_1140A_min,pmts_overdue_1140A_mean,credacc_cards_status_52L_mode,credacc_cards_status_52L_last,last180dayaveragebalance_704A_last,last180dayaveragebalance_704A_mean,last180dayaveragebalance_704A_max
221249,221249,605782,0.000000,-0.003175,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN
1220007,1220007,1814036,0.000000,-0.003175,8.0,NaN,0.100124,0.0,0.0,0.0,0.000000,3254.518,0.0,101.703688,0.0,0.0,NaN,NaN,NaN
1168023,1168023,1762052,0.000000,-0.003175,12.0,39.0,0.100124,0.0,0.0,NaN,1250.000000,0.000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN
145230,145230,190664,0.000000,-0.083811,4.0,42.0,0.100124,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN
338605,338605,723138,0.120749,0.122728,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222458,222458,606991,0.000000,-0.003175,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,4585.916,0.0,1079.536521,0.0,0.0,NaN,NaN,NaN
138401,138401,183835,0.120749,0.122728,8.0,4.8,0.100124,0.0,0.0,0.0,19866.545455,0.000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN
183208,183208,228642,0.000000,-0.083811,1.0,NaN,0.100124,0.0,0.0,NaN,0.000000,3559.084,0.0,131.817926,0.0,0.0,NaN,NaN,NaN
222110,222110,606643,0.000000,-0.003175,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


In [ ]:
ME.evaluate(model.transform_without_predictor(dataset.X_train), dataset.y_train)

c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


Binning 0.114 GB of training data: 0.349 s
Binning 0.013 GB of validation data: 0.010 s
Fitting gradient boosted rounds:
[1/10] 1 tree, 100 leaves, max depth = 13, train loss: 0.68220, val loss: 0.68230, in 0.095s
[2/10] 1 tree, 100 leaves, max depth = 13, train loss: 0.67320, val loss: 0.67354, in 0.100s
[3/10] 1 tree, 100 leaves, max depth = 12, train loss: 0.66576, val loss: 0.66617, in 0.096s
[4/10] 1 tree, 100 leaves, max depth = 12, train loss: 0.65949, val loss: 0.66035, in 0.086s
[5/10] 1 tree, 100 leaves, max depth = 13, train loss: 0.65396, val loss: 0.65523, in 0.096s
[6/10] 1 tree, 100 leaves, max depth = 13, train loss: 0.64915, val loss: 0.65057, in 0.328s
[7/10] 1 tree, 100 leaves, max depth = 14, train loss: 0.64503, val loss: 0.64685, in 0.124s
[8/10] 1 tree, 100 leaves, max depth = 16, train loss: 0.64148, val loss: 0.64362, in 0.112s
[9/10] 1 tree, 100 leaves, max depth = 12, train loss: 0.63832, val loss: 0.64078, in 0.123s
[10/10] 1 tree, 100 leaves, max depth = 15

{'fit_time': array([2.38358784, 2.2126441 , 2.27001071, 2.13887906, 2.12028599]),
 'score_time': array([0.4506042 , 0.66253376, 0.35240984, 0.34714961, 0.36258268]),
 'test_roc_auc_ovr': array([0.69872875, 0.70094035, 0.70105513, 0.69355976, 0.69730474]),
 'test_f1': array([0.1043372 , 0.11056101, 0.10697694, 0.10406319, 0.106513  ])}

In [ ]:
from sklearn.metrics import get_scorer_names
all_scorers = get_scorer_names()
all_scorers

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'neg_root_mean_squared_log_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 '

In [ ]:
from sklearn.metrics import roc_auc_score

def gini(y_true, y_pred):
    return 2 * roc_auc_score(y_true, y_pred) - 1

In [ ]:
gini(dataset.y_train, model.predict_proba(dataset.X_train).T[1])



c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.41753088870334953

In [ ]:
roc_auc_score(dataset.y_train, model.predict_proba(dataset.X_train).T[1])

c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.7087654443516748

In [ ]:
roc_auc_score(dataset.y_oot, model.predict_proba(dataset.X_oot).T[1])

c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in log
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


0.6462233084994017